In [ ]:
# 导入所需模块
import threading
import multiprocessing
import time
import queue
import concurrent.futures
from typing import List, Any
import os

print("线程和多进程环境设置完成！")
print(f"CPU核心数: {multiprocessing.cpu_count()}")
print(f"当前进程ID: {os.getpid()}")


In [ ]:
# 线程基础示例

def worker_function(name: str, work_time: float, shared_counter: List[int]):
    """工作线程函数"""
    print(f"线程 {name} 开始工作")
    
    for i in range(3):
        time.sleep(work_time)
        shared_counter[0] += 1
        print(f"线程 {name} 完成任务 {i+1}, 共享计数器: {shared_counter[0]}")
    
    print(f"线程 {name} 完成工作")

# 1. 创建和启动线程
print("=== 线程基础演示 ===")

# 共享数据
shared_counter = [0]

# 创建线程
threads = []
for i in range(3):
    thread = threading.Thread(
        target=worker_function,
        args=(f"Worker-{i}", 0.5, shared_counter)
    )
    threads.append(thread)

# 启动所有线程
start_time = time.time()
for thread in threads:
    thread.start()

# 等待所有线程完成
for thread in threads:
    thread.join()

end_time = time.time()
print(f"所有线程完成，总耗时: {end_time - start_time:.2f}秒")
print(f"最终计数器值: {shared_counter[0]}")

# 2. 线程池示例
print("\n=== 线程池演示 ===")

def compute_square(n: int) -> int:
    """计算平方数"""
    time.sleep(0.1)  # 模拟计算时间
    result = n * n
    print(f"计算 {n}^2 = {result}")
    return result

# 使用线程池
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    numbers = range(1, 11)
    
    start_time = time.time()
    # 提交任务到线程池
    future_to_number = {executor.submit(compute_square, num): num for num in numbers}
    
    # 获取结果
    results = []
    for future in concurrent.futures.as_completed(future_to_number):
        number = future_to_number[future]
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"计算 {number} 时发生错误: {e}")
    
    end_time = time.time()
    print(f"线程池完成，总耗时: {end_time - start_time:.2f}秒")
    print(f"结果: {sorted(results)}")


In [ ]:
# 线程同步机制示例

# 1. 线程锁 (Lock)
import threading
import time

shared_resource = {"counter": 0}
lock = threading.Lock()

def worker_with_lock(name: str, iterations: int):
    """使用锁的工作函数"""
    for i in range(iterations):
        with lock:
            # 临界区代码
            current_value = shared_resource["counter"]
            time.sleep(0.001)  # 模拟处理时间
            shared_resource["counter"] = current_value + 1
            print(f"线程 {name}: 计数器更新为 {shared_resource['counter']}")

# 2. 条件变量 (Condition)
condition = threading.Condition()
items = []

def producer(name: str, num_items: int):
    """生产者"""
    for i in range(num_items):
        with condition:
            item = f"{name}-item-{i}"
            items.append(item)
            print(f"生产者 {name}: 生产了 {item}")
            condition.notify_all()  # 通知所有等待的消费者
        time.sleep(0.1)

def consumer(name: str, num_items: int):
    """消费者"""
    consumed = 0
    while consumed < num_items:
        with condition:
            while not items:
                print(f"消费者 {name}: 等待商品...")
                condition.wait()  # 等待生产者通知
            
            item = items.pop(0)
            print(f"消费者 {name}: 消费了 {item}")
            consumed += 1
        time.sleep(0.2)

# 3. 信号量 (Semaphore)
semaphore = threading.Semaphore(2)  # 最多2个线程同时访问

def resource_worker(name: str):
    """使用信号量的工作函数"""
    with semaphore:
        print(f"线程 {name}: 获取了资源")
        time.sleep(1)  # 模拟使用资源
        print(f"线程 {name}: 释放了资源")

# 4. 事件 (Event)
event = threading.Event()

def waiter(name: str):
    """等待事件的函数"""
    print(f"线程 {name}: 等待事件...")
    event.wait()  # 等待事件被设置
    print(f"线程 {name}: 接收到事件信号!")

def setter():
    """设置事件的函数"""
    time.sleep(2)
    print("设置事件信号...")
    event.set()

# 演示线程同步
print("=== 线程同步机制演示 ===")

# 1. 测试线程锁
print("1. 线程锁测试:")
shared_resource["counter"] = 0
lock_threads = []

for i in range(3):
    thread = threading.Thread(target=worker_with_lock, args=(f"Worker-{i}", 5))
    lock_threads.append(thread)
    thread.start()

for thread in lock_threads:
    thread.join()

print(f"最终计数器值: {shared_resource['counter']}")

# 2. 测试条件变量
print("\n2. 条件变量测试:")
items.clear()
condition = threading.Condition()

producer_thread = threading.Thread(target=producer, args=("P1", 5))
consumer_thread = threading.Thread(target=consumer, args=("C1", 5))

producer_thread.start()
consumer_thread.start()

producer_thread.join()
consumer_thread.join()

# 3. 测试信号量
print("\n3. 信号量测试:")
semaphore_threads = []

for i in range(5):
    thread = threading.Thread(target=resource_worker, args=(f"Worker-{i}",))
    semaphore_threads.append(thread)
    thread.start()

for thread in semaphore_threads:
    thread.join()

# 4. 测试事件
print("\n4. 事件测试:")
event.clear()  # 重置事件

waiter_threads = []
for i in range(3):
    thread = threading.Thread(target=waiter, args=(f"Waiter-{i}",))
    waiter_threads.append(thread)
    thread.start()

setter_thread = threading.Thread(target=setter)
setter_thread.start()

for thread in waiter_threads:
    thread.join()
setter_thread.join()

print("线程同步演示完成")


In [ ]:
# 多进程编程示例

def cpu_bound_task(n: int) -> int:
    """CPU密集型任务"""
    result = 0
    for i in range(n):
        result += i * i
    return result

def io_bound_task(duration: float) -> str:
    """IO密集型任务"""
    import time
    start_time = time.time()
    time.sleep(duration)
    end_time = time.time()
    return f"任务完成，耗时: {end_time - start_time:.2f}秒"

# 1. 基本多进程
print("=== 基本多进程演示 ===")

if __name__ == "__main__":
    # 创建进程
    processes = []
    numbers = [1000000, 2000000, 1500000]
    
    start_time = time.time()
    
    for i, num in enumerate(numbers):
        process = multiprocessing.Process(
            target=cpu_bound_task,
            args=(num,)
        )
        processes.append(process)
        process.start()
        print(f"启动进程 {i+1}: PID={process.pid}")
    
    # 等待所有进程完成
    for process in processes:
        process.join()
    
    end_time = time.time()
    print(f"多进程完成，总耗时: {end_time - start_time:.2f}秒")

# 2. 进程池
print("\n=== 进程池演示 ===")

def worker_function(args):
    """进程池工作函数"""
    worker_id, task_size = args
    result = cpu_bound_task(task_size)
    return f"Worker {worker_id}: 处理 {task_size} 个数据，结果: {result}"

if __name__ == "__main__":
    # 使用进程池
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        tasks = [(i, 500000) for i in range(8)]
        
        start_time = time.time()
        results = pool.map(worker_function, tasks)
        end_time = time.time()
        
        print(f"进程池完成，耗时: {end_time - start_time:.2f}秒")
        for result in results:
            print(result)

# 3. 进程间通信 - 队列
print("\n=== 进程间通信 - 队列演示 ===")

def producer_process(queue: multiprocessing.Queue, name: str, num_items: int):
    """生产者进程"""
    for i in range(num_items):
        item = f"{name}-item-{i}"
        queue.put(item)
        print(f"生产者 {name}: 生产了 {item}")
        time.sleep(0.1)
    
    # 发送结束信号
    queue.put(None)

def consumer_process(queue: multiprocessing.Queue, name: str):
    """消费者进程"""
    consumed_items = []
    while True:
        item = queue.get()
        if item is None:
            break
        consumed_items.append(item)
        print(f"消费者 {name}: 消费了 {item}")
        time.sleep(0.2)
    
    print(f"消费者 {name}: 总共消费了 {len(consumed_items)} 个项目")

if __name__ == "__main__":
    # 创建队列
    queue = multiprocessing.Queue()
    
    # 创建生产者和消费者进程
    producer = multiprocessing.Process(
        target=producer_process,
        args=(queue, "P1", 5)
    )
    consumer = multiprocessing.Process(
        target=consumer_process,
        args=(queue, "C1")
    )
    
    # 启动进程
    producer.start()
    consumer.start()
    
    # 等待完成
    producer.join()
    consumer.join()

# 4. 进程间通信 - 共享内存
print("\n=== 进程间通信 - 共享内存演示 ===")

def worker_with_shared_memory(shared_array, lock, worker_id: int):
    """使用共享内存的工作进程"""
    for i in range(10):
        with lock:
            # 安全地访问共享内存
            shared_array[worker_id] += 1
            print(f"Worker {worker_id}: 值更新为 {shared_array[worker_id]}")
        time.sleep(0.1)

if __name__ == "__main__":
    # 创建共享数组和锁
    shared_array = multiprocessing.Array('i', [0, 0, 0])  # 3个整数
    lock = multiprocessing.Lock()
    
    # 创建工作进程
    processes = []
    for i in range(3):
        process = multiprocessing.Process(
            target=worker_with_shared_memory,
            args=(shared_array, lock, i)
        )
        processes.append(process)
        process.start()
    
    # 等待所有进程完成
    for process in processes:
        process.join()
    
    print(f"最终共享数组值: {[shared_array[i] for i in range(len(shared_array))]}")

# 5. 线程vs进程性能对比
print("\n=== 线程vs进程性能对比 ===")

def cpu_intensive_work(n: int) -> int:
    """CPU密集型工作"""
    result = 0
    for i in range(n):
        result += i * i
    return result

def compare_threading_vs_multiprocessing():
    """比较线程和进程的性能"""
    task_size = 1000000
    num_workers = 4
    
    # 测试线程性能
    print("测试线程性能...")
    start_time = time.time()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(cpu_intensive_work, task_size) for _ in range(num_workers)]
        thread_results = [future.result() for future in futures]
    
    thread_time = time.time() - start_time
    print(f"线程执行时间: {thread_time:.2f}秒")
    
    # 测试进程性能
    print("测试进程性能...")
    start_time = time.time()
    
    with multiprocessing.Pool(processes=num_workers) as pool:
        process_results = pool.map(cpu_intensive_work, [task_size] * num_workers)
    
    process_time = time.time() - start_time
    print(f"进程执行时间: {process_time:.2f}秒")
    
    print(f"进程相对线程的性能提升: {thread_time/process_time:.2f}倍")

if __name__ == "__main__":
    compare_threading_vs_multiprocessing()

print("\n=== 并发编程总结 ===")
print("""
并发编程的选择指南：
1. 线程适合：I/O密集型任务、需要共享内存的场景
2. 进程适合：CPU密集型任务、需要真正并行计算的场景
3. 异步适合：高并发I/O操作、网络编程
4. 同步机制：锁、信号量、条件变量、事件
5. 进程间通信：队列、管道、共享内存
6. 性能考虑：GIL的影响、上下文切换开销
""")
